In [8]:
import pandas as pd
import unicodedata

In [17]:
def add_space_if_fw(text):
    if unicodedata.east_asian_width(text) in 'FWA':
        return text + ' '
    else:
        return text

In [20]:
def master2df(file_name, widths, names, usecols, fn):

    with open(file_name, 'r', encoding='cp932') as f:
        content = f.read()

    result = ''

    for ch in content:
        result += add_space_if_fw(ch)

    file_name_changed = f'changed_{file_name}.csv'

    with open(file_name_changed, mode='w', encoding='UTF-8') as f:
        f.write(result)
    
    df = pd.read_fwf(file_name_changed, widths=widths, names=names, usecols=usecols, encoding='UTF-8', dtype = str).fillna(fn)

    if file_name == 'USROUT':
        df['sup_name'] = df['sup_name'].str.strip()
    elif file_name == 'KANOUT':
        df = df.astype({'num': int, 'k_num': int, 'y_num': int, 'h_num': int})
        df['seban'] = df['seban'].str.strip()
        df['hinban'] = df['hinban'].str.strip()
        df['store'] = df['store'].str.strip()
        df['box'] = df['box'].str.strip()
        
    return df

In [10]:
def ruiout2df():
    file_name = 'RUIOUT'
    d = 3

    with open(file_name, 'r', encoding='cp932') as f:
        content = ''
        while True:
            line = f.readline()
            if line == '':
                break
            b_line = line[:5]
            st_line = d * 40 + 77 + 1
            a_line = line[-st_line:-78]
            content += b_line + a_line + '\n'

    file_name_changed = f'changed_{file_name}.csv'

    with open(file_name_changed, mode='w', encoding='UTF-8') as f:
        f.write(content)

    widths = [5]
    names = ['ad']
    meisai_width = [6, 2, 1, 6, 2, 4, 5, 3, 6, 5]
    usecols = [0]
    m_cols = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    astype = {}
    for i in range(d):
        widths += meisai_width
        m_name = [f'n_bi{i}', f'n_bin{i}', f'h_kubun{i}', f'h_bi{i}', f'h_bin{i}', f'h_jikan{i}', f'noban{i}', f'hako{i}', f'nonyu{i}', f'aki{i}']
        names += m_name
        usecols += [j + 10 * i for j in m_cols]
        astype[f'hako{i}'] = int
        astype[f'nonyu{i}'] = int

    df = pd.read_fwf(file_name_changed, widths=widths, names=names, usecols=usecols, encoding='UTF-8', dtype = str).fillna(0)
    df = df.astype(astype)
    return df

In [ ]:
#仕入先マスタ
# file_name = 'USROUT'
# widths = [1, 5, 5, 20]
# names = ['aki', 'ad', 'sup_code', 'sup_name']
# usecols= [2, 3]
# fn = 0

#マスタ
# file_name = 'KANOUT'
# widths = [1, 5, 1, 1, 5, 2, 4, 14, 5, 8, 5, 1, 10, 4, 5, 2, 1, 3, 6, 2, 4, 8, 5, 6, 4, 8, 4, 3, 4, 4, 8, 4, 4, 4, 5, 5, 5, 5, 5, 5, 8, 1, 1, 2, 1, 8, 10, 8, 12, 1, 3, 1, 9]
# names = ['aki', 'ad', 'kaitei', 'cp', 'sup_code', 'ukeire', 'seban', 'hinban', 'num', 'store', 'sikyu', 's_kubun', 'line_add', 'aki2', 'shuyoseki', 'tanto', 'iro', 'pocket', 'cycle', 'aki3', 'setteimai', 'setteiryo', 'mai_bin', 'ryo_bin', 'zen_mai', 'zen_ryo', 'k_num', 'y_num', 's_num', 'h_num', 'sohat', 'kinko', 'hakko', 'hakkosumi', 'b_add', 'a_add', 'gai_1', 'gai_2', 'gai_3', 'siharai', 'kigo', 'b_kubun', 'u_kubun', 'mark', 'mark_col', 'box', 'kose', 's_okiba', 'comment', 'sys_kubun', 'shukkaba', 'n_kubun', 'aki4']
# usecols= [1, 4, 6, 7, 8, 9, 26, 27, 29, 45]
# fn = {
#         'num': 0,
#         'k_num': 0,
#         'y_num': 0,
#         'h_num': 0,
#         'ad': '',
#         'sup_code': '',
#         'seban': '',
#         'hinban': '',
#         'store': '',
#         'box': ''
#     }
# df.isnull().any()

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [6]:
# テーブル削除
cur.execute(
    'DROP TABLE IF EXISTS rui'
)

In [ ]:
# マスター作成
cur.execute(
    '''
    CREATE TABLE master (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        sup_code TEXT, 
        seban TEXT, 
        hinban TEXT, 
        num INTEGER, 
        store TEXT, 
        k_num INTEGER, 
        y_num INTEGER, 
        h_num INTEGER,
        box TEXT)
    '''
)						

In [ ]:
# 集欠マスター作成
cur.execute(
    '''
    CREATE TABLE shuketu (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        num INTEGER,
        num_all INTEGER, 
        cust_name TEXT, 
        due_date TEXT, 
        tonyu INTEGER,
        inventory INTEGER, 
        afure INTEGER, 
        shuketubi TEXT, 
        bin INTEGER,
        comment TEXT)
    '''
)		

In [ ]:
# 仕入先マスター作成
cur.execute(
    '''
    CREATE TABLE sup (
        id INTEGER PRIMARY KEY, 
        sup_code TEXT,
        sup_name TEXT)
    '''
)		

In [ ]:
# 累積マスター作成
# cur.execute(
#     '''
#     CREATE TABLE rui (
#         id INTEGER PRIMARY KEY, 
#         ad TEXT,
#         n_bi0 TEXT,
#         n_bin0 TEXT,
#         h_kubun0 TEXT,
#         h_bi0 TEXT,
#         h_bin0 TEXT,
#         h_jikan0 TEXT,
#         noban0 TEXT,
#         hako0 INTEGER,
#         nonyu0 INTEGER,
#         n_bi1 TEXT,
#         n_bin1 TEXT,
#         h_kubun1 TEXT,
#         h_bi1 TEXT,
#         h_bin1 TEXT,
#         h_jikan1 TEXT,
#         noban1 TEXT,
#         hako1 INTEGER,
#         nonyu1 INTEGER,
#         n_bi2 TEXT,
#         n_bin2 TEXT,
#         h_kubun2 TEXT,
#         h_bi2 TEXT,
#         h_bin2 TEXT,
#         h_jikan2 TEXT,
#         noban2 TEXT,
#         hako2 INTEGER,
#         nonyu2 INTEGER
#         )
#     '''
# )

In [7]:
# 累積マスター作成
s = ''
for i in range(3):
    s += f'''
        n_bi{i} TEXT,
        n_bin{i} TEXT,
        h_kubun{i} TEXT,
        h_bi{i} TEXT,
        h_bin{i} TEXT,
        h_jikan{i} TEXT,
        noban{i} TEXT,
        hako{i} INTEGER,
        nonyu{i} INTEGER,'''
s = s[:-1]
ss = f'''
    CREATE TABLE rui (
        id INTEGER PRIMARY KEY, 
        ad TEXT,{s}
    )
    '''
cur.execute(ss)

In [17]:
# マスター→DB
# df.to_sql('master', con, if_exists='append', index=False)

40019

In [9]:
# 仕入先マスター→DB
# df.to_sql('sup', con, if_exists='append', index=False)

596

In [11]:
# 累積マスター→DB
df = ruiout2df()
df.to_sql('rui', con, if_exists='append', index=False)

11

In [ ]:
cur.execute(
    "select * from sqlite_master where type='table' and name='master'"
)
tables = cur.fetchall()
tables

In [81]:
# 全テーブル検索
cur.execute(
    "SELECT NAME FROM sqlite_master WHERE type='table'"
)
tables = cur.fetchall()
tables

[('sqlite_sequence',), ('shuketu',), ('master',), ('sup',), ('rui',)]

In [ ]:
cur.execute(
    "SELECT * FROM master WHERE hinban like '82715%'"
)
tables = cur.fetchall()
tables

In [ ]:
cur.execute(
    "SELECT * FROM rui WHERE id = 3"
)
tables = cur.fetchall()
tables

In [ ]:
cur.execute(
    '''
    INSERT INTO
	    shuketu
    VALUES
        (1,
        '00011',
        1,
        10,
        'aaa',
        '',
        2,
        1,
        1,
        date('now', 'localtime'),
        1,
        '')
    '''
)

In [18]:
con.close()

In [7]:
# モデル設定
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
import datetime
from sqlalchemy import Column, ForeignKey, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

SQLALCHEMY_DATABASE_URL = 'sqlite:///test.db'


engine = create_engine(
  SQLALCHEMY_DATABASE_URL,
  connect_args={'check_same_thread': False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class Master(Base):
  __tablename__ = 'master'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, unique=True, index=True)
  sup_code = Column(String, index=True)
  seban = Column(String, index=True)
  hinban = Column(String, index=True)
  num = Column(Integer, index=True)
  store = Column(String, index=True)
  k_num = Column(Integer, index=True)
  y_num = Column(Integer, index=True)
  h_num = Column(Integer, index=True)
  shuketu = relationship('Shuketu', backref="master")

class Shuketu(Base):
  __tablename__ = 'shuketu'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, ForeignKey('master.ad'), index=True)
  num = Column(Integer, index=True)
  num_all = Column(Integer, index=True)
  cust_name = Column(String, index=True)
  due_date = Column(String, index=True)
  tonyu = Column(Integer, index=True)
  inventory = Column(Integer, index=True)
  afure = Column(Integer, index=True)
  shuketubi = Column(String, index=True)
  bin = Column(Integer, index=True)
  comment = Column(String, index=True)
  

In [8]:
# create
# def create_item(db, args):
#     item = Master(
#         ad=args['ad'], 
#         sup_code=args['sup_code'], 
#         seban=args['seban'], 
#         hinban=args['hinban'], 
#         num=args['num'], 
#         store=args['store'], 
#         k_num=args['k_num'], 
#         y_num=args['y_num'], 
#         h_num=args['h_num'])
#     db.add(item)
#     db.commit()
#     db.refresh(item)
#     return item

def create_item(db, args):

    item = Shuketu(
        ad=args['ad'],
        num=args['num'],
        num_all=args['num_all'],
        cust_name=args['cust_name'],
        due_date=args['due_date'],
        tonyu=args['tonyu'],
        inventory=args['inventory'],
        afure=args['afure'],
        shuketubi=args['shuketubi'],
        bin=args['bin'],
        comment=args['comment'],
    )
    db.add(item)
    db.commit()
    db.refresh(item)
    return item


# args = {}
# args['ad'] = '00001'
# args['sup_code'] = '00100'
# args['seban'] = '010'
# args['hinban'] = '12345-67890-00'
# args['num'] = 100
# args['store'] = 'A01-01-1'
# args['k_num'] = 1
# args['y_num'] = 1 
# args['h_num'] = 1

args = {}
args['ad']= '00010'
args['num']= 1
args['num_all']= 10
args['cust_name']= 'aaa'
args['due_date']= ''
args['tonyu']= 2
args['inventory']= 1
args['afure']= 1
args['shuketubi']= datetime.date.today().isoformat()
args['bin']= 1
args['comment']= ''


db = SessionLocal()
item = create_item(db, args)    
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
db.close()

In [ ]:
# read
# def read_item(db, ad):
#   items = db.query(Master).filter(Master.ad == ad).all()
#   return items

def read_item(db, ad):
  items = db.query(Shuketu).filter(Shuketu.ad == ad).all()
  return items


ad = '00010'
db = SessionLocal()
items = read_item(db, ad)
for item in items:
    # print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
    print(item.id, item.ad, item.num, item.num_all, item.cust_name, item.due_date, item.tonyu, item.inventory, item.afure, item.shuketubi, item.bin, item.comment)
    print(item.master.hinban)
db.close()

In [13]:
# update
# def update_item(db, args):
#     item = db.query(Master).filter(Master.ad == args['ad']).first()
#     item.num = args['num']
#     db.commit()
#     db.refresh(item)
#     return item

def update_item(db, args):
    item = db.query(Shuketu).filter(Shuketu.id == args['id']).first()
    item.num = args['num']
    db.commit()
    db.refresh(item)
    return item

args = {}
args['id'] = 3
args['ad'] = '00001'
args['sup_code'] = '00100'
args['seban'] = '010'
args['hinban'] = '12345-67890-00'
args['num'] = 50000
args['store'] = 'A01-01-1'
args['k_num'] = 1
args['y_num'] = 1 
args['h_num'] = 1

db = SessionLocal()
item = update_item(db, args)
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
print(item.num)
db.close()

50000


In [15]:
# delete
# def delete_item(db, id):
#     item = db.query(Master).filter(Master.id == id).first()
#     db.delete(item)
#     db.commit()

def delete_item(db, id):
    item = db.query(Shuketu).filter(Shuketu.id == id).first()
    db.delete(item)
    db.commit()

id = 3

db = SessionLocal()
delete_item(db, id)
db.close()